In [26]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt
import urllib.request
import os, re, json, random

In [33]:
# 마르코프 체인 딕셔너리 만들기 --- (※1)
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic

# 딕셔너리에 데이터 등록하기 --- (※2)
def set_word3(dic, s3):
    w1, w2, w3 = s3 #w1 ='제' w2='5', w3='편'
    if not w1 in dic: dic[w1] = {} # '제':{}
    if not w2 in dic[w1]: dic[w1][w2] = {} # '제':{'5':{}}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0 # '제':{'5':{'편':{0}}}  차수를 늘려가는 것..[][][] 이 표기법 사용
    dic[w1][w2][w3] += 1

# 문장 만들기 --- (※3)
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic" 
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".": break
        w1, w2 = w2, w3
    ret = "".join(ret)
    # 띄어쓰기
    #params = urllib.parse.urlencode({
   #     "_callback": "",
  #      "q": ret
 #   })
   # 네이버 맞춤법 검사기를 사용합니다. (수정하기-과제)
    #data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/dcontent/spellchecker.nhn?" + params)
    #data = data.read().decode("utf-8")[1:-2]
    #data = json.loads(data)
    #data = data["message"]["result"]["html"]
    #data = soup = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return ret
def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))

def spell(s):
    import requests
    data={}
    data["sentence"]=s.encode("utf-8")
    r = requests.post("https://alldic.daum.net/grammar_checker.do", data=data)
    soup = BeautifulSoup(r.text,'html.parser')
    result=[]
    for text in soup.find_all('a'):
        if not text.get('data-error-output') == None:
            a=text.get('data-error-output')
            result.append(a)
    return(result)

#------main------
# 문장 읽어 들이기 --- (※4)
toji_file = "toji.txt"
dict_file = "markov-toji.json"
if not os.path.exists(dict_file):
    # 토지 텍스트 파일 읽어 들이기
    fp = codecs.open("BEXX0014.txt", "r", encoding="utf-16")
    soup = BeautifulSoup(fp, "html.parser")
    body = soup.select_one("text > body")
    text = body.getText()
    text = text.replace("…", "") # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결 (필요없는거 제거)
                                 # replace: 앞문자>>뒷문자
    # 형태소 분석
    twitter = Okt()
    malist = twitter.pos(text, norm=True) #pos는 품사부착..
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file,"r"))

# 문장 만들기 --- (※6)
for i in range(3):
    s = make_sentence(dic)
    print(s)
    print(spell(s))
    print("---")

얼른화제를잘라버리고홍이먼저방문을잡고있었다.
['얼른 화제를 잘라버리고 홍이 먼저 방문을 잡고 있었다.']
---
용서하세요
심재용은여자니까굶어도함께굶고묵어도함께묵자했지마는임이네도동네에서쬐끼나가지고외지로돌아댕기믄서오만지랄다하고떠들지모를일이었다.
['심재용은 여자니까 굶어도 함께 굶고 묵어도 함께 묵자 했지마는 임이네도동네에서쬐끼나가지고 외지로 돌아다니면 서오만 지랄 다하고 떠들지 모를 일이었다.']
---
사생들만고스란히들어있는병을취급하는야전병원의의사를물어보면서웨이터에게식사를끝내고있는홍이와소사가지연을보고뫼로갈거야.
['사생들만 고스란히 들어있는 병을 취급하는 야전병원의 의사를 물어보면서 웨이터에게 식사를 끝내고 있는 홍이와 소사가 지연을 보고뫼로갈거야.']
---
